In [456]:
!ls proxyattention/runs |wc -l

     445


In [385]:
import os
import pandas as pd
from tensorflow.python.summary.summary_iterator import summary_iterator
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [386]:
def parse_tfevent(tfevent, filepath):
    return dict(
        fname = filepath,
        wall_time=tfevent.wall_time,
        name=tfevent.summary.value[0].tag,
        step=tfevent.step,
        value=tfevent.summary.value[0],
    )


In [387]:
all_files = []
main_path =Path("/Users/eragon/Documents/CODE/Github/improving_robotics_datasets/src/proxyattention/runs")
for (root, _, filenames) in os.walk(main_path):
    for filename in filenames:
        if "events.out.tfevents" in filename:
            all_files.append(root/Path(filename))

In [388]:
full_dict = {}

In [389]:
for files in tqdm(all_files, total = len(all_files)):
    temp_dict = {}
    for ev in summary_iterator(str(files)):
        try:
            parsed = parse_tfevent(ev, files)
            # print(parsed)
            value = parsed["value"]
            # print(value.tensor.string_val[0].decode('ascii'))
            # print("here0")

            # value = value.tag
            try:
                # value = value.tag

                value_str = value.tensor.string_val[0].decode('ascii')
            except:
                try:
                    value_str = value.simple_value[-1]
                except:

                    value_str = value.simple_value
            # print(value)
            # print("here1")
            # try:
            #     temp_dict[value.tag].append(value_str)
            # except Exception as e:
            #     # print(e)
            temp_dict[value.tag.replace("/text_summary", "")] = value_str
        except Exception as e:
            # print(e)
            pass
    full_dict[str(files)] = temp_dict

100%|██████████| 649/649 [00:01<00:00, 465.78it/s]


In [404]:
combined_df = pd.DataFrame.from_records(full_dict).T.reset_index()

In [408]:
combined_df.proxy_steps.unique()

array(["[5, 'p', 5]", 0, '[11]', '[20]', "[10, 'p', 10]", "[2, 'p', 2]",
       '[4]', "[2, 'p', 5]"], dtype=object)

In [407]:
combined_df = combined_df.fillna(0)

In [409]:
combined_df = combined_df[combined_df["proxy_steps"]!=0]

In [411]:
combined_df.proxy_steps.unique()

array(["[5, 'p', 5]", '[11]', '[20]', "[10, 'p', 10]", "[2, 'p', 2]",
       '[4]', "[2, 'p', 5]"], dtype=object)

In [410]:
combined_df.shape

(638, 35)

In [412]:
combined_df.columns

Index(['index', 'experiment_name', 'ds_path', 'ds_name', 'name_fn',
       'image_size', 'batch_size', 'epoch_steps', 'enable_proxy_attention',
       'change_subset_attention', 'num_gpu', 'num_cpu', 'transfer_imagenet',
       'subset_images', 'proxy_threshold', 'pixel_replacement_method', 'model',
       'proxy_steps', 'load_proxy_data', 'gradient_method', 'clear_every_step',
       'fname_start', 'device', 'global_run_count', 'label_map',
       'rev_label_map', 'num_classes', 'proxy_step', 'Loss/Train', 'Acc/Train',
       'Loss/Val', 'Acc/Val', 'Number_Chosen', 'original_images',
       'converted_proxy'],
      dtype='object')

In [413]:
def tensor_str_to_val(string): return np.float32(string.replace("tensor(", "").split(",")[0].strip())

In [414]:
def check_proxy(string): return "p" in string

In [415]:
def calc_stats(values):
    return f"min: {values.min()} \nmax: {values.max()} \navg: {values.mean()}"

In [435]:
combined_df.transfer_imagenet[0]

'True'

In [436]:
def check_wrt_accuracy(has_proxy, condition, head = None):
    grouped_vals = has_proxy.groupby([condition, "transfer_imagenet"]).mean(numeric_only = True)["accuracy"].reset_index().sort_values(by = ["accuracy"], ascending=False)
    print("Transfer")
    if head != None:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == 'True'].head(head).to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == 'False'].head(head).to_markdown())
    else:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == 'True'].to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == 'False'].to_markdown())

## Aggregate results for proxy

In [437]:
combined_df["has_proxy"] = combined_df["proxy_steps"].apply(check_proxy)

In [438]:
combined_df["accuracy"] = combined_df["Acc/Val"]

In [439]:
combined_df["has_proxy"].value_counts()

True     487
False    151
Name: has_proxy, dtype: int64

In [440]:
has_proxy = combined_df[combined_df["has_proxy"] == True]
nothas_proxy = combined_df[combined_df["has_proxy"] == False]

In [441]:
calc_stats(has_proxy["accuracy"])

'min: 0.029623698443174362 \nmax: 0.9905999898910522 \navg: 0.7984235366190606'

In [442]:
calc_stats(nothas_proxy["accuracy"])


'min: 0.030424339696764946 \nmax: 0.9918000102043152 \navg: 0.5552996369017078'

## Parameter elimination

In [443]:
# !python3 -m pip install tabulate

In [467]:
check_wrt_accuracy(combined_df, "clear_every_step")

Transfer
|    | clear_every_step   | transfer_imagenet   |   accuracy |
|---:|:-------------------|:--------------------|-----------:|
|  3 | True               | True                |   0.94145  |
|  1 | False              | True                |   0.938198 |
No Transfer
|    | clear_every_step   | transfer_imagenet   |   accuracy |
|---:|:-------------------|:--------------------|-----------:|
|  2 | True               | False               |  0.0650013 |
|  0 | False              | False               |  0.0634969 |


In [465]:
check_wrt_accuracy(has_proxy, "change_subset_attention")

Transfer
|    |   change_subset_attention | transfer_imagenet   |   accuracy |
|---:|--------------------------:|:--------------------|-----------:|
|  3 |                       0.5 | True                |   0.936635 |
|  5 |                       0.8 | True                |   0.935635 |
|  1 |                       0.3 | True                |   0.93086  |
No Transfer
|    |   change_subset_attention | transfer_imagenet   |   accuracy |
|---:|--------------------------:|:--------------------|-----------:|
|  2 |                       0.5 | False               |  0.0673276 |
|  0 |                       0.3 | False               |  0.0645435 |
|  4 |                       0.8 | False               |  0.0559235 |


In [464]:
check_wrt_accuracy(has_proxy, "gradient_method")

Transfer
|    | gradient_method   | transfer_imagenet   |   accuracy |
|---:|:------------------|:--------------------|-----------:|
|  1 | eigencam          | True                |   0.935007 |
|  5 | gradcamplusplus   | True                |   0.934665 |
|  3 | gradcam           | True                |   0.933682 |
No Transfer
|    | gradient_method   | transfer_imagenet   |   accuracy |
|---:|:------------------|:--------------------|-----------:|
|  4 | gradcamplusplus   | False               |  0.0702486 |
|  0 | eigencam          | False               |  0.059758  |
|  2 | gradcam           | False               |  0.0573011 |


In [463]:
check_wrt_accuracy(has_proxy, "pixel_replacement_method")

Transfer
|    | pixel_replacement_method   | transfer_imagenet   |   accuracy |
|---:|:---------------------------|:--------------------|-----------:|
|  7 | min                        | True                |   0.940078 |
|  3 | max                        | True                |   0.93356  |
|  1 | halfmax                    | True                |   0.933124 |
|  5 | mean                       | True                |   0.930719 |
No Transfer
|    | pixel_replacement_method   | transfer_imagenet   |   accuracy |
|---:|:---------------------------|:--------------------|-----------:|
|  2 | max                        | False               |  0.0663786 |
|  6 | min                        | False               |  0.0648673 |
|  4 | mean                       | False               |  0.0604388 |
|  0 | halfmax                    | False               |  0.060239  |


In [450]:
check_wrt_accuracy(has_proxy, "proxy_threshold", head= 4)

Transfer
|     |   proxy_threshold | transfer_imagenet   |   accuracy |
|----:|------------------:|:--------------------|-----------:|
| 204 |          0.90174  | True                |   0.985    |
| 241 |          0.923236 | True                |   0.9842   |
| 118 |          0.849893 | True                |   0.9838   |
| 103 |          0.843547 | True                |   0.982996 |
No Transfer
|     |   proxy_threshold | transfer_imagenet   |   accuracy |
|----:|------------------:|:--------------------|-----------:|
|  31 |          0.81375  | False               |   0.131932 |
| 177 |          0.883611 | False               |   0.12993  |
| 226 |          0.913836 | False               |   0.126927 |
| 203 |          0.901073 | False               |   0.121048 |


In [451]:

check_wrt_accuracy(has_proxy, "subset_images")

Transfer
|    |   subset_images | transfer_imagenet   |   accuracy |
|---:|----------------:|:--------------------|-----------:|
|  1 |           10000 | True                |   0.966222 |
|  2 |            8000 | True                |   0.903124 |
No Transfer
|    |   subset_images | transfer_imagenet   |   accuracy |
|---:|----------------:|:--------------------|-----------:|
|  0 |           10000 | False               |  0.0628471 |


In [457]:

check_wrt_accuracy(has_proxy, "proxy_steps")

Transfer
|    | proxy_steps   | transfer_imagenet   |   accuracy |
|---:|:--------------|:--------------------|-----------:|
|  2 | [2, 'p', 5]   | True                |   0.967175 |
|  1 | [2, 'p', 2]   | True                |   0.965306 |
|  3 | [5, 'p', 5]   | True                |   0.903124 |
No Transfer
|    | proxy_steps   | transfer_imagenet   |   accuracy |
|---:|:--------------|:--------------------|-----------:|
|  0 | [10, 'p', 10] | False               |  0.0628471 |


In [468]:
check_wrt_accuracy(combined_df, "proxy_steps")

Transfer
|    | proxy_steps   | transfer_imagenet   |   accuracy |
|---:|:--------------|:--------------------|-----------:|
|  1 | [11]          | True                |   0.968625 |
|  3 | [2, 'p', 5]   | True                |   0.967175 |
|  5 | [4]           | True                |   0.966972 |
|  2 | [2, 'p', 2]   | True                |   0.965306 |
|  6 | [5, 'p', 5]   | True                |   0.903124 |
No Transfer
|    | proxy_steps   | transfer_imagenet   |   accuracy |
|---:|:--------------|:--------------------|-----------:|
|  4 | [20]          | False               |  0.065826  |
|  0 | [10, 'p', 10] | False               |  0.0628471 |
